##### KerasNLP로 영어를 스페인어로 번역해보기
##### 만든이 : Abheesht Sharma
##### 최초 작성일 : 2022/05/26
##### 마지막 수정일 : 2022/05/26
##### 설명 : 기계 번역 작업을 위해 sequence-to-sequence Transformer 모델을 학습하기 위해 KerasNLP를 사용합니다.

### 소개
KerasNLP는 NLP와 관련 있는 모델 계층, 토크나이저,  지표 등을 위한 방법들을 제공하고 NLP 파이프라인을 편리하게 구성할 수 있도록 합니다.

이번 예제에서는 KerasNLP 계층을 사용하여 인코더-디코더 Transformer 모델을 구축하고 영어-스페인어 기계 번역 작업에서 학습합니다.

(fchollet)[https://twitter.com/fchollet]의 (영어-스페인어 NMT 예)(https://keras.io/examples/nlp/neural_machine_translation_with_transformer/)를 기반으로 합니다. 원래 예제는 더 낮은 수준이고 처음부터 레이어를 구현하는 반면, 이 예제는 KerasNLP를 사용하여 하위 단어 토큰화 및 지표를 사용하여 생성한 번역의 품질을 계산하는 것과 같은 몇 가지 고급 접근 방식을 보여줍니다.

다음 방법을 배우게 됩니다.

* [keras_nlp.tokenizers.WordPieceTokenizer](https://keras.io/api/keras_nlp/tokenizers/word_piece_tokenizer#wordpiecetokenizer-class)를 사용하여 텍스트를 토큰화합니다.
* KerasNLP의 [keras_nlp.layers.TransformerEncoder](https://keras.io/api/keras_nlp/layers/transformer_encoder#transformerencoder-class), [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class) 및 [keras_nlp.layers.TokenAndPositionEmbedding](https://keras.io/api/keras_nlp/layers/token_and_position_embedding#tokenandpositionembedding-class) 레이어를 사용하여 sequence-to-sequence Transformer 모델을 구현하고 학습합니다.
* [keras_nlp.utils.greedy_search](https://keras.io/api/keras_nlp/utils/greedy_search#greedysearch-function) 함수를 사용하여 탐욕스러운 디코딩 전략을 사용하여 보이지 않는 입력 문장의 번역을 생성하십시오!

KerasNLP에 익숙하지 않더라도 걱정하지 마십시오. 이 튜토리얼은 기초부터 시작합니다. 바로 시작하겠습니다!

### 사전 준비 (Setup)

파이프라인을 시작하기 전에 필요한 모든 라이브러리를 불러옵니다.

In [1]:
!pip install -q rouge-score

In [2]:
import keras_nlp
import numpy as np
import pathlib
import random
import tensorflow as tf

from tensorflow import keras
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

파라이터 및 하이퍼 파라미터를 정의합니다.

In [3]:
BATCH_SIZE = 64
EPOCHS = 1  # 수렴 (convergence)를 위해서는 최소한 10으로 설정해야 합니다.
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

데이터 다운드

Anki[https://www.manythings.org/anki/]가 제공하는 영어-스페인어 번역 데이터셋을 사용합니다. 다운로드 해보겠습니다:

In [4]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

데이터 전처리

행마다 영어 문장과 스페인어 문자을 포함하고 있습니다. 영어 문장이 입력이고, 스페인어 문장이 출력입니다. 각 문장들을 소문자로 변환합니다.

In [5]:
with open(text_file, encoding = "UTF-8") as f: # encoding = "UTF-8" 추가
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = eng.lower()
    spa = spa.lower()
    text_pairs.append((eng, spa))

처리한 결과는 다음과 같습니다:

In [6]:
for _ in range(5):
    print(random.choice(text_pairs))

('tom asked mary if she had seen john.', 'tom preguntó a mary si había visto a john.')
('the dog kept barking all night.', 'el perro siguió ladrando toda la noche.')
('i want a cup of tea.', 'quiero una taza de té.')
('my watch loses two minutes a day.', 'mi reloj se retrasa dos minutos al día.')
("i don't really understand what you want to say.", 'en realidad no entiendo lo que quieres decir.')


이제 데이터를 학습데이터, 검증데이터, 테스트 데이터로 분리하겠습니다.

In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


데이터 토큰화 하기 (Tokenizing)

문장을 토큰할 때 2가지 tokenizer를 사용합니다. 입력인 영어와 출력인 스페인어에 대해서 각각입니다. 텍스트를 토큰화하기 위해 [keras_nlp.tokenizers.WordPieceTokenizer](https://keras.io/api/keras_nlp/tokenizers/word_piece_tokenizer#wordpiecetokenizer-class)를 사용합니다. [keras_nlp.tokenizers.WordPieceTokenizer](https://keras.io/api/keras_nlp/tokenizers/word_piece_tokenizer#wordpiecetokenizer-class)는 WordPiece Vocabulary와 텍스트를 토큰화하거나 다시 문장으로 만들 함수가 필요합니다.

tokenizer 2개를 정의하기 전에, 준비한 데이터셋에 학습을 시켜야 합니다. WordPiece 토큰화 알고리즘은 subword 토큰화 알고리즘입니다. 말뭉치 (corpus)로 학습을 시키면 subword vocabulary을 얻습니다. 하위 단어 토크나이저는 단어 토크나이저(단어 토크나이저는 입력 단어를 잘 다루기 위해 매우 큰 어휘가 필요함)와 문자 토크나이저(문자는 실제로 단어처럼 의미를 인코딩하지 않음) 사이의 절충안입니다. TensorFlow Text를 사용하면 [가이드](https://www.tensorflow.org/text/guide/subwords_tokenizer)에 설명된 대로 말뭉치에서 WordPiece를 매우 간단하게 학습할 수 있습니다.

In [8]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    bert_vocab_args = dict(
        # The target vocabulary size
        vocab_size=vocab_size,
        # Reserved tokens that must be included in the vocabulary
        reserved_tokens=reserved_tokens,
        # Arguments for `text.BertTokenizer`
        bert_tokenizer_params={"lower_case": True},
    )

    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = bert_vocab.bert_vocab_from_dataset(
        word_piece_ds.batch(1000).prefetch(2), **bert_vocab_args
    )
    return vocab

모든 어휘에는 몇 가지 특별하고 예약된 토큰이 있습니다. 다음과 같은 4가지 토큰이 있습니다.

* "[PAD]" - 패딩 토큰. 입력 시퀀스 길이가 최대 시퀀스 길이보다 짧은 경우 패딩 토큰이 입력 시퀀스 길이에 추가됩니다.
* "[UNK]" - 알 수 없는 토큰입니다.
* "[START]" - 입력 시퀀스의 시작을 표시하는 토큰입니다.
* "[END]" - 입력 시퀀스의 끝을 표시하는 토큰입니다.

In [9]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

spa_samples = [text_pair[1] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)

토큰들을 확인해 봅시다!

In [10]:
print("English Tokens: ", eng_vocab[100:110])
print("Spanish Tokens: ", spa_vocab[100:110])

English Tokens:  ['how', 'time', 'll', 'very', 'as', 'did', 'all', 'had', 'here', 'up']
Spanish Tokens:  ['tengo', 'quiero', 'aqui', 'cuando', 'casa', '##n', 'hacer', 'puedo', 'todo', 'esto']


이제 토크나이저를 정의해 보겠습니다. 위에서 훈련된 어휘로 토크나이저를 구성할 것입니다.

In [11]:
eng_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
spa_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False
)

데이터 세트에서 샘플을 토큰화해 봅시다! 텍스트가 올바르게 토큰화되었는지 확인하기 위해 토큰 목록을 원래 텍스트로 다시 토큰화할 수도 있습니다.

In [12]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print("Recovered text after detokenizing: ", eng_tokenizer.detokenize(eng_tokens_ex))

print()

spa_input_ex = text_pairs[0][1]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("Spanish sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print("Recovered text after detokenizing: ", spa_tokenizer.detokenize(spa_tokens_ex))

English sentence:  i will have him come.
Tokens:  tf.Tensor([ 34  98  69  92 130  11], shape=(6,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'i will have him come .', shape=(), dtype=string)

Spanish sentence:  yo lo haré venir.
Tokens:  tf.Tensor([ 90  73 692 371  13], shape=(5,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'yo lo hare venir .', shape=(), dtype=string)


### 데이터세트 형식
다음으로 데이터세트의 형식을 지정합니다.

각 훈련 단계에서 모델은 소스 문장과 대상 단어 0에서 N을 사용하여 대상 단어 N+1(및 그 이상)을 예측하려고 합니다.

따라서 훈련 데이터 세트는 튜플(입력, 대상)을 생성합니다. 여기서:

* 입력은 인코더_입력 및 디코더_입력 키가 있는 사전입니다. 인코더_입력은 토큰화된 소스 문장이고 디코더_입력은 "지금까지" 대상 문장, 즉 대상 문장에서 단어 N+1(및 그 이후)을 예측하는 데 사용되는 단어 0에서 N입니다.
* 출력은 한 단계 오프셋된 대상 문장입니다. 대상 문장의 다음 단어, 즉 모델이 예측하려고 시도할 단어를 제공합니다.

텍스트를 토큰화한 후 입력된 스페인어 문장에 "[START]" 및 "[END]" 특수 토큰을 추가합니다. 또한 입력을 고정 길이로 채웁니다. 이것은 keras_nlp.[layers.StartEndPacker](https://keras.io/api/keras_nlp/layers/start_end_packer#startendpacker-class)를 사용하여 쉽게 수행할 수 있습니다.

In [13]:
def preprocess_batch(eng, spa):
    batch_size = tf.shape(spa)[0]

    eng = eng_tokenizer(eng)
    spa = spa_tokenizer(spa)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `spa` and pad it as well.
    spa_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=spa_tokenizer.token_to_id("[START]"),
        end_value=spa_tokenizer.token_to_id("[END]"),
        pad_value=spa_tokenizer.token_to_id("[PAD]"),
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )

In [14]:
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

In [15]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

시퀀스 모양을 간단히 살펴보겠습니다(64쌍의 배치가 있고 모든 시퀀스의 길이는 40단계입니다).

In [16]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


### 모델 구축
이제 모델을 정의하는 흥미로운 부분으로 넘어가겠습니다! 먼저 임베딩 레이어, 즉 입력 시퀀스의 모든 토큰에 대한 벡터가 필요합니다. 이 임베딩 레이어는 무작위로 초기화될 수 있습니다. 시퀀스의 단어 순서를 인코딩하는 위치 임베딩 레이어도 필요합니다. 관례는 이 두 개의 임베딩을 추가하는 것입니다. KerasNLP에는 위의 모든 단계를 수행하는 keras_nlp.layers.TokenAndPositionEmbedding 레이어가 있습니다.

우리의 sequence-to-sequence Transformer는 함께 연결된 [keras_nlp.layers.TransformerEncoder](https://keras.io/api/keras_nlp/layers/transformer_encoder#transformerencoder-class) 레이어와 [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class) 레이어로 구성됩니다.

소스 시퀀스는 [keras_nlp.layers.TransformerEncoder](https://keras.io/api/keras_nlp/layers/transformer_encoder#transformerencoder-class)에 전달되어 새로운 표현을 생성합니다. 그런 다음 이 새로운 표현은 지금까지의 대상 시퀀스(대상 단어 0에서 N까지)와 함께 [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class)로 전달됩니다. 그런 다음 [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class)는 대상 시퀀스의 다음 단어(N+1 이상)를 예측하려고 합니다.

이를 가능하게 하는 주요 세부 사항은 인과 관계 마스킹입니다. [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class)는 전체 시퀀스를 한 번에 확인하므로 토큰 N+1을 예측할 때 대상 토큰 0에서 N까지의 정보만 사용하도록 해야 합니다(그렇지 않으면 미래의 정보를 사용할 수 있으므로 결과적으로 추론 시점에 사용할 수 없는 모델에서). 인과 마스킹은 [keras_nlp.layers.TransformerDecoder](https://keras.io/api/keras_nlp/layers/transformer_decoder#transformerdecoder-class)에서 기본적으로 활성화되어 있습니다.

패딩 토큰("[PAD]")도 마스킹해야 합니다. 이를 위해 [keras_nlp.layers.TokenAndPositionEmbedding](https://keras.io/api/keras_nlp/layers/token_and_position_embedding#tokenandpositionembedding-class) 레이어의 mask_zero 인수를 True로 설정할 수 있습니다. 그런 다음 모든 후속 레이어에 전파합니다.

In [17]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SPA_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

d:\workspace\lecture_keras\venv\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


### 모델 훈련
유효성 검사 데이터에 대한 학습 진행 상황을 빠르게 모니터링하는 방법으로 정확도를 사용합니다. 기계 번역은 일반적으로 정확도보다는 BLEU 점수와 기타 측정항목을 사용합니다. 그러나 ROUGE, BLEU 등과 같은 지표를 사용하려면 확률을 디코딩하고 텍스트를 생성해야 합니다. 텍스트 생성은 계산 비용이 많이 들고 훈련 중에 이 작업을 수행하는 것은 권장하지 않습니다.

예제에서는 1 epoch 동안만 훈련하지만 모델이 실제로 수렴되도록 하려면 최소한 10 epoch 동안 훈련해야 합니다.

In [18]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 token_and_position_embedding (  (None, None, 256)   3850240     ['encoder_inputs[0][0]']         
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   1315072     ['token_and_position_em

### 시험문장 복호화(정성분석)
마지막으로 새로운 영어 문장을 번역하는 방법을 보여드리겠습니다. 토큰화된 영어 문장과 대상 토큰 "[START]"을 모델에 입력하기만 하면 됩니다. 모델은 다음 토큰의 확률을 출력합니다. 그런 다음 토큰 "[END]"에 도달할 때까지 지금까지 생성된 토큰을 조건으로 다음 토큰을 반복적으로 생성했습니다.

디코딩을 위해 KerasNLP의 [keras_nlp.utils.greedy_search](https://keras.io/api/keras_nlp/utils/greedy_search#greedysearch-function) 함수를 사용합니다. Greedy Decoding은 각 시간 단계에서 가장 가능성이 높은 다음 토큰, 즉 확률이 가장 높은 토큰을 출력하는 텍스트 디코딩 방법입니다.

In [19]:
def decode_sequences(input_sentences):
    batch_size = tf.shape(input_sentences)[0]

    # Tokenize the encoder input.
    encoder_input_tokens = eng_tokenizer(input_sentences).to_tensor(
        shape=(None, MAX_SEQUENCE_LENGTH)
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]

    # Set the prompt to the "[START]" token.
    prompt = tf.fill((batch_size, 1), spa_tokenizer.token_to_id("[START]"))

    generated_tokens = keras_nlp.utils.greedy_search(
        token_probability_fn,
        prompt,
        max_length=40,
        end_token_id=spa_tokenizer.token_to_id("[END]"),
    )
    generated_sentences = spa_tokenizer.detokenize(generated_tokens)
    return generated_sentences


test_eng_texts = [pair[0] for pair in test_pairs]
for i in range(2):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequences(tf.constant([input_sentence]))
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

** Example 0 **
i loved it in boston.
me gustaria en boston .

** Example 1 **
i like the blue one. how much does it cost?
¿ me gusta el mundo como un poco que el es un buen ?



### 모델 평가(정량적 분석)
텍스트 생성 작업에 사용되는 많은 지표가 있습니다. 여기에서 우리 모델에 의해 생성된 번역을 평가하기 위해 ROUGE-1 및 ROUGE-2 점수를 계산해 보겠습니다. 기본적으로 ROUGE-N은 참조 텍스트와 생성된 텍스트 사이의 공통 n-gram 수를 기반으로 한 점수입니다. ROUGE-1과 ROUGE-2는 각각 공통 유니그램 수와 바이그램 수를 사용합니다.

30개의 테스트 샘플에 대해 점수를 계산할 것입니다(디코딩은 비용이 많이 드는 과정입니다).

In [20]:
rouge_1 = keras_nlp.metrics.RougeN(order=1)
rouge_2 = keras_nlp.metrics.RougeN(order=2)

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequences(tf.constant([input_sentence]))
    translated_sentence = translated_sentence.numpy()[0].decode("utf-8")
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    rouge_1(reference_sentence, translated_sentence)
    rouge_2(reference_sentence, translated_sentence)

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())

ROUGE-1 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.3027645>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.30783066>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.29881045>}
ROUGE-2 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.11721141>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.12122175>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.11641605>}
